<center><h1>Datos Meteorológicos con Métodos de Interpolación</h1></center>  
<center><img src="clima.png" width=270 height=270></center>  

### **Sección: 305C1**
### **Estudiantes:**
Manuel Nava 30.822.007  
Juan Wu 30.391.117  
José Farrauto 30.696.288  

---

## Interpolación de Taylor

---

## Interpolación de Lagrange

In [32]:
#=Importar libreria necesaria=#
using PyCall
using Plots
np = pyimport("numpy")

#=Datos de prueba=#
baja = np.array([10,7,4,3,5,2,2]) #=Temperatura baja=# 
alta = np.array([21,18,12,11,14,13,10]) #=Temperatura alta=#

#=Variables para trabajar:=#
dias = length(baja) #Esto es para contar cuanto dias
promedio_T = np.zeros(dias)
dias_lista = np.zeros(dias)

#Sacar promedio de temperatura
for i in 1:dias
    promedio_T[i]= (alta[i]+baja[i])/2
end

#=Contar cantidad de dias=#
j = 1
for i in 1:dias
    dias_lista[i] = j
    j = j + 1
end

#Procedimiento para el Interpolacion de Lagrange:

#Sacar el valor el vector "X":
menordias = np.min(dias_lista)
mayordias = np.max(dias_lista)
xpp = np.linspace(menordias, mayordias, 100)

100-element Vector{Float64}:
 1.0
 1.0606060606060606
 1.121212121212121
 1.1818181818181819
 1.2424242424242424
 1.303030303030303
 1.3636363636363638
 1.4242424242424243
 1.4848484848484849
 1.5454545454545454
 ⋮
 6.515151515151516
 6.575757575757576
 6.636363636363637
 6.696969696969697
 6.757575757575758
 6.818181818181818
 6.878787878787879
 6.9393939393939394
 7.0

---

## Interpolación de Hermite

---

## Interpolación Polinómicas a Trozos

<center><h2>Gracias!!</h2></center>